# Imports



In [16]:
import pandas as pd
import requests
import re
import numpy as np
import sqlite3

from datetime import datetime 
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

# DataCollection

In [2]:
# Parameter
headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}

# Url
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# Request
page = requests.get(url, headers=headers)

# Beautiful soup object
soup = BeautifulSoup(page.text, 'html.parser')

#=============================== Product Data ===============================
products = soup.find('ul', class_='products-listing small')

# List class
products_list = soup.find_all('article', class_='hm-product-item')

# product_id
product_id = [p.get('data-articlecode') for p in products_list]

#product_category
product_category = [p.get('data-category') for p in products_list]


products_listb = products.find_all('a', class_='link')

# product_name
product_name = [p.get_text() for p in products_listb]

# List price
products_listc = products.find_all('span', class_='price regular')

# product_price
product_price = [p.get_text() for p in products_listc]

# Unifque 
data = pd.DataFrame([product_id, product_category, product_name, product_price]).T

# Name columns
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']


#  DataCollection by product

In [3]:

# Empty dataframe
df_compositions = pd.DataFrame()

# unique columns for all products
aux = []

df_pattern = pd.DataFrame(columns=['Art. No.', 'Composition', 'Fit', 'Size', 'Product safety'])

for i in range(len(data)):
     # buscar  a url
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    print('Produto: {}'.format(url))

    page = requests.get(url, headers=headers)

    # Beautiful soup object
    soup = BeautifulSoup(page.text, 'html.parser')

     # ==================== product_color ======================
    product_list = soup.find_all('a', class_='filter-option miniature active') + soup.find_all('a', class_='filter-option miniature')
   # loop for 
    color_name = [p.get('data-color') for p in product_list]

    # product_id
    product_id = [p.get('data-articlecode') for p in product_list]

    # cria o dataframe
    df_color = pd.DataFrame([product_id, color_name]).T 
    df_color.columns = ['product_id', 'color_name']
    
    # tadas as cores
    for j in range(len(df_color)):

        # buscar  a url
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'
        print('Color: {}'.format(url))

        page = requests.get(url, headers=headers)

        # Beautiful soup object
        soup = BeautifulSoup(page.text, 'html.parser')
        
        # ==================== product_name ====================
        product_name = soup.find_all('h1', class_='primary product-item-headline')
        product_name = product_name[0].get_text()
        
        # ==================== produc_price ====================
        product_price = soup.find_all('div', class_='primary-row product-item-price')
        product_price = re.findall(r'\d+\.?\d+', product_price[0].get_text())[0]
        
        # ==================== composition  ======================
        product_composition_list = soup.find_all('div', class_='pdp-description-list-item')

        # loop for
        product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

        # criar o dataframe e localizar a primeira linha
        df_composition = pd.DataFrame(product_composition).T
        df_composition.columns = df_composition.iloc[0]

        df_composition = df_composition.iloc[1:].fillna(method='ffill')

        # Remove poket lining, shell and lining   <<<===========================================================
        df_composition['Composition'] = df_composition['Composition'].replace('Pocket lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].replace('Pocket: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].replace('Lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].replace('Shell: ', '', regex=True)

        # concatenar as colunas do df_composition com df_pattern 
        df_composition = pd.concat([df_pattern, df_composition], axis=0)

        # raname columns 
        df_composition.columns = ['product_id','composition', 'fit', 'size', 'product_safety' ]
        df_composition['product_name'] = product_name
        df_composition['product_pric'] = product_price

        # alguma nova coluna diferente guardar no aux
        aux = aux + df_composition.columns.tolist()

        # marge
        df_composition = pd.merge(df_composition, df_color, how='left', on='product_id')

        # concatenar final
        df_compositions = pd.concat([df_compositions, df_composition], axis=0)

    
# Join showroom data + style
df_compositions['style_id'] = df_compositions['product_id'].apply(lambda x: x[:-3])
df_compositions['color_id'] = df_compositions['product_id'].apply(lambda x: x[-3:])   

#scrapy datetime
df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


Produto: https://www2.hm.com/en_us/productpage.0636207011.html
Color: https://www2.hm.com/en_us/productpage.0636207011.html
Color: https://www2.hm.com/en_us/productpage.0636207001.html
Color: https://www2.hm.com/en_us/productpage.0636207002.html
Color: https://www2.hm.com/en_us/productpage.0636207004.html
Color: https://www2.hm.com/en_us/productpage.0636207005.html
Color: https://www2.hm.com/en_us/productpage.0636207006.html
Color: https://www2.hm.com/en_us/productpage.0636207010.html
Color: https://www2.hm.com/en_us/productpage.0636207014.html
Color: https://www2.hm.com/en_us/productpage.0636207015.html
Color: https://www2.hm.com/en_us/productpage.0636207017.html
Color: https://www2.hm.com/en_us/productpage.0636207019.html
Produto: https://www2.hm.com/en_us/productpage.0720504010.html
Color: https://www2.hm.com/en_us/productpage.0720504010.html
Color: https://www2.hm.com/en_us/productpage.0720504001.html
Color: https://www2.hm.com/en_us/productpage.0720504002.html
Color: https://www2.

Color: https://www2.hm.com/en_us/productpage.0811993003.html
Color: https://www2.hm.com/en_us/productpage.0811993006.html
Color: https://www2.hm.com/en_us/productpage.0811993007.html
Color: https://www2.hm.com/en_us/productpage.0811993021.html
Color: https://www2.hm.com/en_us/productpage.0811993022.html
Color: https://www2.hm.com/en_us/productpage.0811993024.html
Color: https://www2.hm.com/en_us/productpage.0811993025.html
Color: https://www2.hm.com/en_us/productpage.0811993028.html
Color: https://www2.hm.com/en_us/productpage.0811993031.html
Color: https://www2.hm.com/en_us/productpage.0811993034.html
Color: https://www2.hm.com/en_us/productpage.0811993037.html
Produto: https://www2.hm.com/en_us/productpage.0730863033.html
Color: https://www2.hm.com/en_us/productpage.0730863033.html
Color: https://www2.hm.com/en_us/productpage.0730863002.html
Color: https://www2.hm.com/en_us/productpage.0730863003.html
Color: https://www2.hm.com/en_us/productpage.0730863004.html
Color: https://www2.hm

Color: https://www2.hm.com/en_us/productpage.0811993021.html
Color: https://www2.hm.com/en_us/productpage.0811993022.html
Color: https://www2.hm.com/en_us/productpage.0811993024.html
Color: https://www2.hm.com/en_us/productpage.0811993025.html
Color: https://www2.hm.com/en_us/productpage.0811993028.html
Color: https://www2.hm.com/en_us/productpage.0811993031.html
Color: https://www2.hm.com/en_us/productpage.0811993034.html
Color: https://www2.hm.com/en_us/productpage.0811993036.html
Produto: https://www2.hm.com/en_us/productpage.0690449022.html
Color: https://www2.hm.com/en_us/productpage.0690449022.html
Color: https://www2.hm.com/en_us/productpage.0690449001.html
Color: https://www2.hm.com/en_us/productpage.0690449002.html
Color: https://www2.hm.com/en_us/productpage.0690449006.html
Color: https://www2.hm.com/en_us/productpage.0690449007.html
Color: https://www2.hm.com/en_us/productpage.0690449009.html
Color: https://www2.hm.com/en_us/productpage.0690449011.html
Color: https://www2.hm

# Data Cleanig

In [4]:
#     product_category
df_data = df_compositions.dropna(subset=['product_id'])

#     product_name
df_data['product_name'] = df_data['product_name'].str.replace('\n', '')
df_data['product_name'] = df_data['product_name'].str.replace('\t', '')
df_data['product_name'] = df_data['product_name'].str.replace('  ', '')
df_data['product_name'] = df_data['product_name'].str.replace(' ', '_').str.lower()

# product_price
df_data['product_pric'] = df_data['product_pric'].astype(float)

#     color_name
df_data['color_name'] = df_data['color_name'].str.replace(' ', '_').str.lower()

#     Fit
df_data['fit'] = df_data['fit'].apply(lambda x: x.replace(' ', '_').lower())

#     Size
df_data['size_number'] = df_data['size'].apply(lambda x: re.search('\d{3}', x).group(0) if pd.notnull(x) else x)

#     Size Model
df_data['size_model'] = df_data['size'].str.extract('(\d+/\d+)')

#     DropSize 
df_data = df_data.drop(columns=['size'], axis=1)

#     break composition
df1 = df_data['composition'].str.split(',', expand=True).reset_index(drop=True)

# # =======================    Composition    =======================
# Cotton | Polyester | Elasterell-P | Elastane
df_ref = pd.DataFrame(index=np.arange(len(df_data)),columns=['cotton', 'polyester', 'elastane', 'elasterell'])

# # ===========================================================================================
# ---------------->> Cotton
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton', na=True), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains('Cotton', na=True), 1]
df_cotton_1.name = 'cotton'

# combine cotton
df_cotton = df_cotton_0.combine_first(df_cotton_1)

# concatenar
df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]


# ---------------->> Polyester
df_polyester_0 = df1.loc[df1[0].str.contains('Polyester', na=True), 0]
df_polyester_0.name = 'polyester'
df_polyester_1 = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester_1.name = 'polyester'

# combine
df_polyester = df_polyester_0.combine_first(df_polyester_1)
# concatenar
df_ref = pd.concat([df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]


# ---------------->> Elastane
df_Elastane_1 = df1.loc[df1[1].str.contains('Elastane', na=True), 1]
df_Elastane_1.name = 'elastane'
df_Elastane_2 = df1.loc[df1[2].str.contains('Elastane', na=True), 2]
df_Elastane_2.name = 'elastane'
df_Elastane_3 = df1.loc[df1[3].str.contains('Elastane', na=True), 3]
df_Elastane_3.name = 'elastane'

# combine
df_Elastane_c = df_Elastane_1.combine_first(df_Elastane_2)
df_elastane = df_Elastane_c.combine_first(df_Elastane_3)

# concatenar
df_ref = pd.concat([df_ref, df_elastane], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]


# ---------------->> Elasterell-P
df_Elasterell = df1.loc[df1[1].str.contains('Elasterell-P', na=True), 1]
df_Elasterell.name = 'elasterell'

# concatenar
df_ref = pd.concat([df_ref, df_Elasterell], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]


# join of combine with product_id
df_aux = pd.concat([df_data['product_id'].reset_index(drop=True), df_ref], axis=1)

# deixar as porcentagens nas composições
df_aux['cotton'] = df_aux['cotton'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
df_aux['polyester'] = df_aux['polyester'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
df_aux['elasterell'] = df_aux['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
df_aux['elastane'] = df_aux['elastane'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)

# # ===========================================================================================
# # final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna(0)

# merge
df_data = pd.merge(df_data, df_aux, on='product_id', how='left')
# # ===========================================================================================

# deixar so as colunas
df_data = df_data.drop(columns=['composition'], axis=1)

# drop duplicate
df_data = df_data.drop_duplicates()

In [7]:
df_data


,product_id,fit,product_safety,product_name,product_pric,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,elastane,elasterell
0,0636207011,slim_fit,NaN,slim_jeans,19.99,midnight_blue,0636207,011,2021-09-06 14:25:08,NaN,NaN,0.89,0.65,0.01,0.0
2,0636207001,slim_fit,NaN,slim_jeans,19.99,dark_denim_blue,0636207,001,2021-09-06 14:25:08,NaN,NaN,0.89,0.10,0.01,0.0
3,0636207002,slim_fit,NaN,slim_jeans,5.99,dark_gray_denim,0636207,002,2021-09-06 14:25:08,NaN,NaN,0.66,0.28,0.01,0.0
4,0636207004,slim_fit,NaN,slim_jeans,10.99,denim_blue,0636207,004,2021-09-06 14:25:08,NaN,NaN,0.88,0.10,0.02,0.0
5,0636207005,slim_fit,NaN,slim_jeans,11.99,gray,0636207,005,2021-09-06 14:25:08,NaN,NaN,1.00,0.24,0.01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0427159017,skinny_fit,NaN,trashed_skinny_jeans,24.99,super_light_denim_blue,0427159,017,2021-09-06 14:25:08,NaN,NaN,1.00,0.00,0.00,0.0
449,0427159022,skinny_fit,NaN,trashed_skinny_jeans,9.99,black/trashed,0427159,022,2021-09-06 14:25:08,NaN,NaN,0.93,0.06,0.01,0.0
450,0427159023,skinny_fit,NaN,trashed_skinny_jeans,21.99,light_denim_blue,0427159,023,2021-09-06 14:25:08,NaN,NaN,0.93,0.06,0.01,0.0
451,0427159025,skinny_fit,NaN,trashed_skinny_jeans,14.99,denim_blue/bleached,0427159,025,2021-09-06 14:25:08,NaN,NaN,0.93,0.06,0.01,0.0


# Data Insert

In [9]:
data_insert = df_data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'color_name',
    'fit',
    'product_pric',
    'size_number',
    'size_model',
    'cotton',
    'polyester',
    'elastane',
    'elasterell',
    'scrapy_datetime'    
]] 

In [12]:
# query_showroom_schema = """
#     CREATE TABLE vitrine(
#         product_id        TEXT,
#         style_id          TEXT,
#         color_id          TEXT,
#         product_name      TEXT,
#         color_name        TEXT,
#         fit               TEXT,
#         product_pric      REAL,
#         size_number       TEXT,
#         size_model        TEXT,
#         cotton            REAL,
#         polyester         REAL,
#         elastane          REAL,
#         elasterell        REAL,
#         scrapy_datetime   TEXT   
#     )
# """


In [13]:
# create table
conn = sqlite3.connect('database_hm.sqlite')
cursor = conn.execute(query_showroom_schema)
conn.commit()

In [14]:
# create database conection
conn = create_engine('sqlite:///database_hm.sqlite', echo=False)

# data insert
data_insert.to_sql('vitrine', con=conn, if_exists='append', index=False)

In [15]:
# table view
query = """
    SELECT *
    FROM vitrine
"""

df = pd.read_sql_query(query, conn)
df.head()

,product_id,style_id,color_id,product_name,color_name,fit,product_pric,size_number,size_model,cotton,polyester,elastane,elasterell,scrapy_datetime
0,0636207011,0636207,011,slim_jeans,midnight_blue,slim_fit,19.99,None,None,0.89,0.65,0.01,0.0,2021-09-06 14:25:08
1,0636207001,0636207,001,slim_jeans,dark_denim_blue,slim_fit,19.99,None,None,0.89,0.10,0.01,0.0,2021-09-06 14:25:08
2,0636207002,0636207,002,slim_jeans,dark_gray_denim,slim_fit,5.99,None,None,0.66,0.28,0.01,0.0,2021-09-06 14:25:08
3,0636207004,0636207,004,slim_jeans,denim_blue,slim_fit,10.99,None,None,0.88,0.10,0.02,0.0,2021-09-06 14:25:08
4,0636207005,0636207,005,slim_jeans,gray,slim_fit,11.99,None,None,1.00,0.24,0.01,0.0,2021-09-06 14:25:08
